# Packaging and Deploying our ML model to our edge devices - Using IoT jobs

In this exercise, we will:

 - Invoke SageMaker NEO to compile the trained model 
 - Create a deployment package with a signed model + the runtime used by SageMaker Edge Agent to load and invoke the optimized model
 - Deploy the package using IoT Jobs

In [ ]:
import sagemaker
import time
import boto3
import json
import uuid

s3_client = boto3.client('s3')
sm_client = boto3.client('sagemaker')

sagemaker_session = sagemaker.Session()

role = sagemaker.get_execution_role()
bucket_name = 'sagemaker-wind-turbine-farm-%s' % project_id

prefix='wind_turbine_anomaly'

### Compiling our ML model

### Invoking SageMaker NEO to compile the trained model

In [ ]:
compilation_job_name = 'wind-turbine-anomaly-%d' % int(time.time()*1000)
sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': '%s%s/output/model.tar.gz' % (estimator_output_path, training_job_name),
        'DataInputConfig': '{"input0":[1,%d,10,10]}' % n_features,
        'Framework': 'PYTORCH'
    },
    OutputConfig={
        'S3OutputLocation': 's3://%s/wind_turbine/optimized/' % sagemaker_session.default_bucket(),        
        'TargetPlatform': { 'Os': 'LINUX', 'Arch': 'X86_64' }
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 900 }
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)

### Building the Deployment Package SageMaker Edge Manager
It will sign the model and create a deployment package with:
 - The optimized model
 - Model Metadata
 - SageMaker NEO runtime (dlr)

In [ ]:
model_version = '1.0'
model_name = 'WindTurbineAnomalyDetection'
edge_packaging_job_name='wind-turbine-anomaly-%d' % int(time.time()*1000)

resp = sm_client.create_edge_packaging_job(
    EdgePackagingJobName=edge_packaging_job_name,
    CompilationJobName=compilation_job_name,
    ModelName=model_name,
    ModelVersion=model_version,
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': 's3://%s/%s/model/' % (bucket_name, prefix)
    }
)
while True:
    resp = sm_client.describe_edge_packaging_job(EdgePackagingJobName=edge_packaging_job_name)    
    if resp['EdgePackagingJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['EdgePackagingJobStatus'], compilation_job_name)
        break
    time.sleep(5)

### Deploy the package
Using IoT Jobs, we will notify the Python application in the edge devices. The application will:
 - download the deployment package
 - unpack it
 - load the new mode (unload previous versions if any)

In [ ]:
iot_client = boto3.client('iot')
sts_client = boto3.client('sts')

region_name = sagemaker_session.boto_session.region_name
account_id = sts_client.get_caller_identity()["Account"]


In [ ]:
resp = iot_client.create_job(
    jobId=str(uuid.uuid4()),
    targets=[
        'arn:aws:iot:%s:%s:thinggroup/WindTurbineFarm-%s' % (region_name, account_id, project_id),        
    ],
    document=json.dumps({
        'type': 'new_model',
        'model_version': model_version,
        'model_name': model_name,
        'model_package_bucket': bucket_name,
        'model_package_key': "%s/model/%s-%s.tar.gz" % (prefix, model_name, model_version)        
    }),
    targetSelection='SNAPSHOT'
)

First, let us restore variables stored in the previous exercise

In [ ]:
%store -r project_id
%store -r estimator_output_path
%store -r training_job_name
%store -r n_features

Alright! Now, the deployment process will start on the connected edge devices! 
You can start the Exercise #4: Run a simulated fleet of wind turbines and edge devices. Predict anomalies
 
 > [Exercise 04](../../04-Run-Fleet/greengrass-v2/04-run-fleet-iot-jobs.ipynb)
